# First test of the modeling part of the pipeline: responsible for creating the model
Notebook structure (recommended)
2. Data loading: load preprocessed datasets and metadata.
4. Model: define, train, persist.
5. Evaluation: compute and save metrics and plots.
6. Save artifacts: model, transformers, metrics, config.

In [1]:
import pandas as pd
import numpy as np

df_train = pd.read_csv('../data/churn/train.csv')

In [2]:
# Define the specific dates range
start_date = "2023-06-01"
end_date = "2023-09-30"

safra = []
# Generate a random date within the specified range
for i in range(df_train.shape[0]):
    safra.append(
        pd.to_datetime(
            np.random.choice(pd.date_range(start=start_date, end=end_date))
        ).strftime("%Y%m")
    )

df_train["safra"] = safra

rng = np.random.RandomState(42)
n_ones = int(round(0.2 * len(df_train)))

df_train['no_action'] = 0
df_train.loc[rng.choice(df_train.index, size=n_ones, replace=False), 'no_action'] = 1


In [3]:
# mark categorical features so LightGBM can handle them natively
for c in ['Geography', 'Gender']:
    if c in df_train.columns:
        df_train[c] = df_train[c].astype('category')

In [4]:
# create a new categorical column with 3 random values using the existing RNG
choices = ['Abcon', 'Jacli', 'Pcos']
df_train['canal'] = pd.Categorical(rng.choice(choices, size=len(df_train)))

# quick check
df_train['canal'].value_counts()

canal
Abcon    55055
Jacli    55003
Pcos     54976
Name: count, dtype: int64

In [5]:
# create a new categorical column with 3 random values using the existing RNG
choices = ['Bco1', 'Bco2', 'Bco3', 'Bco4', 'Bco5']
df_train['bcos'] = pd.Categorical(rng.choice(choices, size=len(df_train)))

In [6]:
# create a new categorical column with 4 random values using the existing RNG
choices = ['tgt', 'tgt_excred', 'tgt_exp', 'ntgt']
df_train['class_tgt'] = pd.Categorical(rng.choice(choices, size=len(df_train)))

In [7]:
df_oot = df_train[df_train['safra'] == '202309']
df_train = df_train[df_train['safra'] != '202309']


### 1. Mdl Training

In [8]:
pipeline_parameters = {
    'description_general': '''

    <div style="display:grid; grid-template-columns:300px 1fr; gap:8px 16px; align-items:start;">
        <div class="txt-hg-blue">Projeto:</div>
        <div class="txt-hg-bold">Ariel - Concessão de Crédito Rotativo</div>

        <div class="txt-hg-blue">Objetivo:</div>
        <div>Desenvolver modelo auxiliar target na tomada de decisão.</div>

        <div class="txt-hg-blue">Target:</div>
        <div><span style="background:#ffe8e0; color:#7a2b15; padding:3px 8px; border-radius:12px; font-weight:600;">Over 60 Mob 6</span></div>

        <div class="txt-hg-blue">Período de Treinamento:</div>
        <div>2024-06 a 2024-08</div>

        <div class="txt-hg-blue">Período de OOT:</div>
        <div>2024-09 a 2024-10</div>

        <div class="txt-hg-blue">Métricas de Avaliação:</div>
        <div>Gini Coefficient, % Alto Risco Target, % Baixo Risco Não Target</div>
      </div>

    ''',
    'target_obs': 'no_action',
    'target_obs_inf': 'no_action',
    'no_action_var': 'no_action',
    'date_var': 'safra',
    'date_oot': ['202406', '202407', '202408'],
    'features_excluded': ['ID', 'safra', 'no_action'],
    'mdls': [],
    'target_mdl': '',
    'categorical_features': [],
    'description_specific': '',
    'model_name': '',
    'tbl_version': '',
    'inf_version': '',
    'mdl_version': '',
}


pipeline_parameters['description_specific'] = 'Modelo inicial sem seleção de features e sem otimizacao de parametros.'

pipeline_parameters['tbl_version'] = 'ariel_tbl_0_0_1'
pipeline_parameters['inf_version'] = 'ariel_inf_0_0_1'
pipeline_parameters['mdl_version'] = 'sand_0_0_1'

pipeline_parameters['model_name'] = 'ariel_mdl_0_0_1'


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report
import lightgbm as lgb

# Train a LightGBM classifier for churn prediction (new notebook cell)


# feature setup: drop identifiers and text surname
drop_cols = ['id', 'CustomerId', 'Surname', 'safra', 'no_action']
target = 'Exited'

X = df_train.drop(columns=drop_cols + [target])
y = df_train[target]


# train/validation split
X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# model
model = lgb.LGBMClassifier(random_state=42)

# fit with early stopping
model.fit(
    X_tr, y_tr,
    eval_set=[(X_val, y_val)],
    eval_metric='auc',
    categorical_feature=['Geography', 'Gender']
)


# validation metrics
val_probs = model.predict_proba(X_val)[:, 1]
val_preds = (val_probs >= 0.5).astype(int)
print("Validation AUC:", round(roc_auc_score(y_val, val_probs), 4))
print("Validation Accuracy:", round(accuracy_score(y_val, val_preds), 4))
print(classification_report(y_val, val_preds, digits=4))

[LightGBM] [Info] Number of positive: 20953, number of negative: 78442
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000957 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 871
[LightGBM] [Info] Number of data points in the train set: 99395, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.210805 -> initscore=-1.320078
[LightGBM] [Info] Start training from score -1.320078
Validation AUC: 0.8855
Validation Accuracy: 0.8654
              precision    recall  f1-score   support

           0     0.8890    0.9477    0.9174     19611
           1     0.7400    0.5571    0.6357      5238

    accuracy                         0.8654     24849
   macro avg     0.8145    0.7524    0.7766     24849
weighted avg     0.8576    0.8654    0.8580     24849



In [10]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report
import lightgbm as lgb

# Train a LightGBM classifier for churn prediction (new notebook cell)


# feature setup: drop identifiers and text surname
drop_cols = ['id', 'CustomerId', 'Surname', 'safra', 'no_action']
target = 'Exited'

X = df_train.drop(columns=drop_cols + [target])
y = df_train[target]

# train/validation split
X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.2, random_state=32, stratify=y)

# model
model2 = lgb.LGBMClassifier(random_state=32)

# fit with early stopping
model2.fit(
    X_tr, y_tr,
    eval_set=[(X_val, y_val)],
    eval_metric='auc',
    categorical_feature=['Geography', 'Gender']
)


# validation metrics
val_probs = model2.predict_proba(X_val)[:, 1]
val_preds = (val_probs >= 0.5).astype(int)
print("Validation AUC:", round(roc_auc_score(y_val, val_probs), 4))
print("Validation Accuracy:", round(accuracy_score(y_val, val_preds), 4))
print(classification_report(y_val, val_preds, digits=4))

[LightGBM] [Info] Number of positive: 20953, number of negative: 78442
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000922 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 871
[LightGBM] [Info] Number of data points in the train set: 99395, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.210805 -> initscore=-1.320078
[LightGBM] [Info] Start training from score -1.320078
Validation AUC: 0.8914
Validation Accuracy: 0.868
              precision    recall  f1-score   support

           0     0.8919    0.9476    0.9189     19611
           1     0.7441    0.5701    0.6456      5238

    accuracy                         0.8680     24849
   macro avg     0.8180    0.7588    0.7822     24849
weighted avg     0.8608    0.8680    0.8613     24849



In [11]:
df_train['model_score'] = model.predict_proba(X)[:, 1]
df_train['model_score2'] = model2.predict_proba(X)[:, 1]

#### 1.1 Compute Gini Graph

In [12]:
def gini_from_scores(y_true, y_score):
    try:
        auc = roc_auc_score(y_true, y_score)
    except Exception:
        return np.nan
    return 2 * auc - 1

mdls = ['model_score']

# Gini per safra

gini_per_safra = pd.DataFrame()

for mdl in mdls:
    gini_safra = (
        df_train.groupby("safra")
        .apply(lambda g: gini_from_scores(g[target], g[mdl]))
        .reset_index()
        .rename(columns={0: f"gini_{mdl}"})
    )
    if gini_per_safra.empty:
        gini_per_safra = gini_safra
    else:
        gini_per_safra = gini_per_safra.merge(gini_safra, on="safra")

        
    gini_per_safra[f"gini_{mdl}"] = (gini_per_safra[f"gini_{mdl}"] * 100).round(2)


gini_per_safra

/tmp/ipykernel_31935/970451252.py:17: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: gini_from_scores(g[target], g[mdl]))


,safra,gini_model_score
0,202306,79.37
1,202307,79.51
2,202308,79.56


In [13]:
import altair as alt
    
gini_cols = [c for c in gini_per_safra.columns if c.startswith("gini_")]
y_max = gini_per_safra[gini_cols].max(axis=0).values.max()
y_min = gini_per_safra[gini_cols].min(axis=0).values.min()

y_scale = alt.Scale(domain=[max(0, y_min - 5), y_max + 5], nice=True)

line = (
    alt.Chart(gini_per_safra)
    .mark_line(point=True)
    .encode(
        x=alt.X("safra:N", title="Safra", sort="ascending", axis=alt.Axis(labelAngle=-45)),
        y=alt.Y("gini_model_score:Q", title="Gini", scale=y_scale, axis=alt.Axis(labels=False)),
    )
)

labels = (
    alt.Chart(gini_per_safra)
    .mark_text(align="center", dy=-10, color="black")
    .encode(
        x=alt.X("safra:N", sort="ascending"),
        y=alt.Y("gini_model_score:Q", scale=y_scale),
        text=alt.Text("gini_model_score:Q", format=".1f"),
    )
)

chart_line = (line + labels).properties(width=200, height=250)

chart_line


alt.LayerChart(...)

In [14]:
# build gini per safra for both models and plot them on the same chart
mdls = ['model_score', 'model_score2']

def create_gini_chart(df_train, target, mdls):
        
    # compute gini per safra for each model
    gini_per_safra = (
        df_train.groupby("safra")
        .apply(lambda g: pd.Series({f"gini_{m}": 2*roc_auc_score(g[target], g[m]) - 1 for m in mdls}))
        .reset_index()
    )

    for col in [f"gini_{m}" for m in mdls]:
        gini_per_safra[col] = (gini_per_safra[col] * 100).round(2)

    # long format for Altair
    gini_long = gini_per_safra.melt(id_vars="safra", value_vars=[f"gini_{m}" for m in mdls],
                                    var_name="model", value_name="gini")

    # y scale domain
    y_max = gini_long["gini"].max()
    y_min = gini_long["gini"].min()
    y_scale = alt.Scale(domain=[max(0, y_min - 5), y_max + 5], nice=True)
    # line + points per model with legend
    line = (
        alt.Chart(gini_long)
        .mark_line(point=True)
        .encode(
            x=alt.X("safra:N", title="Safra", sort="ascending", axis=alt.Axis(labelAngle=-45)),
            y=alt.Y("gini:Q", title="Gini", scale=y_scale),
            color=alt.Color("model:N", title="Model", legend=alt.Legend(orient="bottom")),
            tooltip=["safra", "model", alt.Tooltip("gini:Q", format=".2f")],
        )
    )

    # labels on points (no legend duplication) with smaller font
    labels = (
        alt.Chart(gini_long)
        .mark_text(align="center", dy=-10, fontSize=9)
        .encode(
            x=alt.X("safra:N", sort="ascending"),
            y=alt.Y("gini:Q", scale=y_scale),
            text=alt.Text("gini:Q", format=".1f"),
            color=alt.Color("model:N", legend=None),
        )
    )

    (chart := (line + labels).properties(width=200, height=150))
    return chart

gini_obs_graph = create_gini_chart(df_train, target='Exited', mdls=mdls)
gini_obs_inf_graph = create_gini_chart(df_train, target='Exited', mdls=mdls)

temp = df_train[df_train['no_action'] == 1]
gini_no_action_graph = create_gini_chart(temp, target='Exited', mdls=mdls)

/tmp/ipykernel_31935/4103862584.py:9: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({f"gini_{m}": 2*roc_auc_score(g[target], g[m]) - 1 for m in mdls}))
/tmp/ipykernel_31935/4103862584.py:9: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({f"gini_{m}": 2*roc_auc_score(g[target], g[m]) - 1 for m in mdls}))
/tmp/ipykernel_31935/4103862584.py:9: FutureWarning: DataFrame

In [15]:
gini_obs_graph_json = gini_obs_graph.to_json()
gini_obs_inf_graph_json = gini_obs_inf_graph.to_json()
gini_no_action_graph_json = gini_no_action_graph.to_json()

In [16]:
gini_train = round((2*roc_auc_score(df_train['Exited'], df_train['model_score']) - 1) * 100, 2)

df_val = df_train[df_train.index.isin(X_val.index)]
gini_test = round((2*roc_auc_score(df_val['Exited'], model.predict_proba(df_val[model.feature_names_in_])[:, 1]) - 1) * 100, 2)

gini_oot = round((2*roc_auc_score(df_oot['Exited'], model.predict_proba(df_oot[model.feature_names_in_])[:, 1]) - 1) * 100, 2)
gini_no_action = round((2*roc_auc_score(df_train[df_train['no_action'] == 1]['Exited'], df_train[df_train['no_action'] == 1]['model_score']) - 1) * 100, 2)

#### 1.1 Geral - Compute Gini matrix per segments

In [17]:
# compute Gini (2*AUC-1) per group and overall, store in gini_df
rows = []
groups = ['bcos', 'canal']

# helper to compute gini safely (returns percent * 100)
def safe_gini(y_true, y_score):
    try:
        if y_true.nunique() < 2:
            return np.nan
        g = 2 * roc_auc_score(y_true, y_score) - 1
        return round(g * 100, 2)
    except Exception:
        return np.nan

# overall
overall_row = {
    "level_type": "overall",
    "n": len(df_train),
    "segment": "all",
}
for m in mdls:
    overall_row[f"gini_{m}"] = safe_gini(df_train[target], df_train[m])
rows.append(overall_row)


for grp in groups:
    for level in df_train[grp].cat.categories:
        row = {
            "level_type": grp,
            "n": len(df_train[df_train[grp]==level]),
            "segment": level,
        }
        for m in mdls:
            row[f"gini_{m}"] = safe_gini(
                df_train[df_train[grp]==level][target],
                df_train[df_train[grp]==level][m]
            )
        rows.append(row)

gini_df = pd.DataFrame(rows)


In [18]:
# transpose so models are rows and each segment is a column
gini_long = gini_df.melt(
    id_vars=['level_type', 'segment'],
    value_vars=[c for c in gini_df.columns if c.startswith('gini_')],
    var_name='model',
    value_name='gini'
)
gini_long['model'] = gini_long['model'].str.replace('gini_', '')

gini_matrix = gini_long.pivot(index='model', columns='segment', values='gini')

# order columns: overall 'all' first, then bcos then canal (if present)
cols_order = ['all'] \
    + [seg for grp in groups for seg in gini_df[gini_df['level_type']==grp]['segment'].tolist()]
cols_order = [c for c in cols_order if c in gini_matrix.columns]
gini_matrix = gini_matrix[cols_order]

gini_obs_seg_json = gini_matrix.to_html(border=0)

In [19]:
# add a row with counts "n" for each segment as the first row, expressed as percentage of the total
n_row = gini_df.set_index('segment')['n'].reindex(cols_order)
total = n_row.loc['all'] if 'all' in n_row.index else n_row.sum()
n_pct = (n_row / total * 100).round(2).astype(str) + '%'
n_pct.name = 'Obs + Inferred'

n_row_geral = n_pct.to_frame().T
#vol_seg_json = n_row.to_frame().T.to_html(border=0)

#### 1.1 Obs - Compute Gini matrix per segments

In [20]:
# compute Gini (2*AUC-1) per group and overall, store in gini_df
rows = []
groups = ['bcos', 'canal']

# helper to compute gini safely (returns percent * 100)
def safe_gini(y_true, y_score):
    try:
        if y_true.nunique() < 2:
            return np.nan
        g = 2 * roc_auc_score(y_true, y_score) - 1
        return round(g * 100, 2)
    except Exception:
        return np.nan

# Change target to 'Exited'
target = 'Exited'
temp = df_train[df_train['Exited'].isin([1,0])]

# overall
overall_row = {
    "level_type": "overall",
    "n": len(temp),
    "segment": "all",
}
for m in mdls:
    overall_row[f"gini_{m}"] = safe_gini(temp[target], temp[m])
rows.append(overall_row)


for grp in groups:
    for level in temp[grp].cat.categories:
        row = {
            "level_type": grp,
            "n": len(temp[temp[grp]==level]),
            "segment": level,
        }
        for m in mdls:
            row[f"gini_{m}"] = safe_gini(
                temp[temp[grp]==level][target],
                temp[temp[grp]==level][m]
            )
        rows.append(row)

gini_df = pd.DataFrame(rows)


In [21]:
# transpose so models are rows and each segment is a column
gini_long = gini_df.melt(
    id_vars=['level_type', 'segment'],
    value_vars=[c for c in gini_df.columns if c.startswith('gini_')],
    var_name='model',
    value_name='gini'
)
gini_long['model'] = gini_long['model'].str.replace('gini_', '')

gini_matrix = gini_long.pivot(index='model', columns='segment', values='gini')

# order columns: overall 'all' first, then bcos then canal (if present)
cols_order = ['all'] \
    + [seg for grp in groups for seg in gini_df[gini_df['level_type']==grp]['segment'].tolist()]
cols_order = [c for c in cols_order if c in gini_matrix.columns]
gini_matrix = gini_matrix[cols_order]

gini_obs_inf_seg_json = gini_matrix.to_html(border=0)

In [22]:
# add a row with counts "n" for each segment as the first row, expressed as percentage of the total
n_row = gini_df.set_index('segment')['n'].reindex(cols_order)
n_pct = (n_row / total * 100).round(2).astype(str) + '%'
n_pct.name = 'Obs'

n_row_obs = n_pct.to_frame().T
#vol_seg_json = n_row.to_frame().T.to_html(border=0)

#### 1.1 No Action - Compute Gini matrix per segments

In [23]:
# compute Gini (2*AUC-1) per group and overall, store in gini_df
rows = []
groups = ['bcos', 'canal']

# helper to compute gini safely (returns percent * 100)
def safe_gini(y_true, y_score):
    try:
        if y_true.nunique() < 2:
            return np.nan
        g = 2 * roc_auc_score(y_true, y_score) - 1
        return round(g * 100, 2)
    except Exception:
        return np.nan


temp = df_train[df_train['no_action'] == 1]
    
# overall
overall_row = {
    "level_type": "overall",
    "n": len(temp),
    "segment": "all",
}

for m in mdls:
    overall_row[f"gini_{m}"] = safe_gini(temp[target], temp[m])
rows.append(overall_row)


for grp in groups:
    for level in temp[grp].cat.categories:
        row = {
            "level_type": grp,
            "n": len(temp[temp[grp]==level]),
            "segment": level,
        }
        for m in mdls:
            row[f"gini_{m}"] = safe_gini(
                temp[temp[grp]==level][target],
                temp[temp[grp]==level][m]
            )
        rows.append(row)

gini_df = pd.DataFrame(rows)


In [24]:
# transpose so models are rows and each segment is a column
gini_long = gini_df.melt(
    id_vars=['level_type', 'segment'],
    value_vars=[c for c in gini_df.columns if c.startswith('gini_')],
    var_name='model',
    value_name='gini'
)
gini_long['model'] = gini_long['model'].str.replace('gini_', '')

gini_matrix = gini_long.pivot(index='model', columns='segment', values='gini')

# order columns: overall 'all' first, then bcos then canal (if present)
cols_order = ['all'] \
    + [seg for grp in groups for seg in gini_df[gini_df['level_type']==grp]['segment'].tolist()]
cols_order = [c for c in cols_order if c in gini_matrix.columns]
gini_matrix = gini_matrix[cols_order]

gini_no_action_seg_json = gini_matrix.to_html(border=0)

In [25]:
# add a row with counts "n" for each segment as the first row, expressed as percentage of the total
n_row = gini_df.set_index('segment')['n'].reindex(cols_order)
n_pct = (n_row / total * 100).round(2).astype(str) + '%'
n_pct.name = 'No Action'

n_row_no_action = n_pct.to_frame().T
#vol_seg_json = n_row.to_frame().T.to_html(border=0)

In [26]:
volume_df = pd.concat([n_row_geral ,n_row_obs, n_row_no_action])

In [27]:
vol_seg_json = volume_df.to_html(border=0)

In [28]:
vol_seg_json = vol_seg_json.replace(
    'class="dataframe"',
    'class="dataframe no-color"'
)

#### 1.1 Compute Pivot tbl tgt vs mdl decil

Volume per class_tgt and model_score decile

In [29]:
# create decile (1 = top decile by model_score, 10 = bottom)
df_train['decile_model_score'] = pd.qcut(df_train['model_score'].rank(method='first'), 10, labels=False)

# pivot: counts of class_tgt per decile
pivot_counts = pd.pivot_table(
    df_train,
    index='decile_model_score',
    columns='class_tgt',
    values='model_score',
    aggfunc='count',
    fill_value=0
).sort_index()

# add total column
pivot_counts['total'] = pivot_counts.sum(axis=1)

# pivot: row-wise percentages
pivot_pct = pivot_counts.div(pivot_counts['total'], axis=0).drop(columns=['total']).round(2) * 100

# total per class (across all deciles)
total_per_class = pivot_counts.drop(columns=['total']).sum(axis=0)
pct_total_per_class = (total_per_class / total_per_class.sum() * 100).round(2)

# add a "Total" row with overall class percentages
pivot_pct.loc['Total'] = pct_total_per_class

# include the total per decile as a new column
pivot_pct['Total %'] = (pivot_counts['total'] / pivot_counts['total'].sum() * 100).round(2)
pivot_pct.loc['Total', 'Total %'] = 100.0

pivot_pct


/tmp/ipykernel_31935/9034436.py:5: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot_counts = pd.pivot_table(


class_tgt,ntgt,tgt,tgt_excred,tgt_exp,Total %
decile_model_score,,,,,
0,25.00,25.00,25.00,25.00,10.0
1,25.00,25.00,25.00,25.00,10.0
2,25.00,25.00,25.00,26.00,10.0
3,26.00,26.00,24.00,25.00,10.0
4,25.00,25.00,25.00,25.00,10.0
5,25.00,26.00,25.00,25.00,10.0
6,25.00,26.00,24.00,25.00,10.0
7,26.00,25.00,25.00,25.00,10.0
8,25.00,25.00,25.00,25.00,10.0


In [30]:
vol_class_tgt_json = pivot_pct.to_html(border=0)

In [31]:
vol_class_tgt_json = vol_class_tgt_json.replace(
    'class="dataframe"',
    'class="dataframe no-color"'
)

In [32]:
temp = df_train[df_train['no_action'] == 1]

# create decile (1 = top decile by model_score, 10 = bottom)
temp['decile_model_score'] = pd.qcut(temp['model_score'].rank(method='first'), 10, labels=False)

# pivot: counts of class_tgt per decile
pivot_counts = pd.pivot_table(
    temp,
    index='decile_model_score',
    columns='class_tgt',
    values='model_score',
    aggfunc='count',
    fill_value=0
).sort_index()

# add total column
pivot_counts['total'] = pivot_counts.sum(axis=1)

# pivot: row-wise percentages
pivot_pct = pivot_counts.div(pivot_counts['total'], axis=0).drop(columns=['total']).round(2) * 100

# total per class (across all deciles)
total_per_class = pivot_counts.drop(columns=['total']).sum(axis=0)
pct_total_per_class = (total_per_class / total_per_class.sum() * 100).round(2)

# add a "Total" row with overall class percentages
pivot_pct.loc['Total'] = pct_total_per_class

# include the total per decile as a new column
pivot_pct['Total %'] = (pivot_counts['total'] / pivot_counts['total'].sum() * 100).round(2)
pivot_pct.loc['Total', 'Total %'] = 100.0

pivot_pct


/tmp/ipykernel_31935/4003351434.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['decile_model_score'] = pd.qcut(temp['model_score'].rank(method='first'), 10, labels=False)
/tmp/ipykernel_31935/4003351434.py:7: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot_counts = pd.pivot_table(


class_tgt,ntgt,tgt,tgt_excred,tgt_exp,Total %
decile_model_score,,,,,
0,24.00,25.00,25.00,26.0,10.0
1,25.00,24.00,26.00,25.0,10.0
2,26.00,25.00,25.00,24.0,10.0
3,25.00,26.00,24.00,25.0,10.0
4,24.00,24.00,27.00,26.0,10.0
5,24.00,27.00,24.00,25.0,10.0
6,24.00,26.00,24.00,26.0,10.0
7,26.00,25.00,25.00,24.0,10.0
8,24.00,26.00,25.00,25.0,10.0


In [33]:
vol_no_action_class_tgt_json = pivot_pct.to_html(border=0)

In [34]:
vol_no_action_class_tgt_json = vol_no_action_class_tgt_json.replace(
    'class="dataframe"',
    'class="dataframe no-color"'
)

%Exited per class_tgt and model_score decile

In [35]:
# create decile (1 = top decile by model_score, 10 = bottom)
df_train['decile_model_score'] = pd.qcut(df_train['model_score'], 10, labels=False)

# pivot: counts of class_tgt per decile
pivot_counts = pd.pivot_table(
    df_train,
    index='decile_model_score',
    columns='class_tgt',
    values='Exited',
    aggfunc='mean',
    fill_value=0
).sort_index().round(4) * 100


total_per_decil = df_train.groupby('decile_model_score')['Exited'].mean().round(4) * 100
total_per_class = df_train.groupby('class_tgt')['Exited'].mean().round(4) * 100

total_per_class.loc['Total'] = df_train['Exited'].mean().round(4) * 100
 
pivot_counts['Total'] = total_per_decil
pivot_counts.loc['Total'] = total_per_class

pivot_counts


/tmp/ipykernel_31935/853188257.py:5: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot_counts = pd.pivot_table(
/tmp/ipykernel_31935/853188257.py:16: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  total_per_class = df_train.groupby('class_tgt')['Exited'].mean().round(4) * 100


class_tgt,ntgt,tgt,tgt_excred,tgt_exp,Total
decile_model_score,,,,,
0,0.49,0.32,0.67,0.64,0.53
1,1.93,1.04,1.45,1.60,1.51
2,2.25,2.23,1.99,2.22,2.17
3,4.37,4.13,4.54,3.70,4.19
4,5.87,6.84,5.51,6.09,6.08
5,11.31,11.19,10.96,10.24,10.93
6,17.50,17.03,17.68,16.90,17.27
7,29.69,30.53,30.34,28.81,29.85
8,52.54,52.87,52.02,53.22,52.66


In [36]:
df_train['decile_model_score'] = pd.qcut(df_train['model_score'], 10, labels=False)

# 2) aggregate sum and count of Exited per (decile, class)
agg = (
    df_train
    .groupby(['decile_model_score', 'class_tgt'])['Exited']
    .agg(['sum', 'count'])
    .unstack('class_tgt')  # columns: multiindex (sum, class), (count, class)
)

# separate sum and count tables, fill missing with 0
sum_tbl = agg['sum'].fillna(0)
cnt_tbl = agg['count'].fillna(0)

# 3) cumulative sums over deciles (index order is decile_model_score)
cum_sum = sum_tbl.cumsum()   # cumulative exited by decile × class
cum_cnt = cnt_tbl.cumsum()   # cumulative population by decile × class

# 4) cumulative exited mean per decile × class
pivot_cum_mean = (cum_sum / cum_cnt * 100).round(2)

# 5) cumulative "Total" per decile (across all classes)
decile_sum_total = sum_tbl.sum(axis=1)
decile_cnt_total = cnt_tbl.sum(axis=1)

cum_sum_total = decile_sum_total.cumsum()
cum_cnt_total = decile_cnt_total.cumsum()

total_per_decil_cum = (cum_sum_total / cum_cnt_total * 100).round(2)

pivot_cum_mean['Total'] = total_per_decil_cum

# 6) final "Total" row (overall mean per class and overall)
total_per_class = df_train.groupby('class_tgt')['Exited'].mean().round(2) * 100
total_per_class.loc['Total'] = df_train['Exited'].mean().round(2) * 100

pivot_cum_mean.loc['Total'] = total_per_class

pivot_cum_mean

/tmp/ipykernel_31935/2438545052.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(['decile_model_score', 'class_tgt'])['Exited']
/tmp/ipykernel_31935/2438545052.py:34: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  total_per_class = df_train.groupby('class_tgt')['Exited'].mean().round(2) * 100


class_tgt,ntgt,tgt,tgt_excred,tgt_exp,Total
decile_model_score,,,,,
0,0.49,0.32,0.67,0.64,0.53
1,1.21,0.68,1.06,1.12,1.02
2,1.56,1.20,1.37,1.49,1.40
3,2.28,1.95,2.13,2.03,2.10
4,3.00,2.93,2.81,2.83,2.89
5,4.39,4.33,4.17,4.05,4.23
6,6.26,6.16,6.07,5.89,6.10
7,9.25,9.17,9.13,8.71,9.06
8,14.09,14.05,13.87,13.61,13.91


In [37]:
mean_class_tgt_json = pivot_cum_mean.to_html(border=0)

In [38]:
mean_class_tgt_json = mean_class_tgt_json.replace(
    'class="dataframe"',
    'class="dataframe invert-color"'
)

In [39]:
temp = df_train[df_train['no_action'] == 1]

# create decile (1 = top decile by model_score, 10 = bottom)
temp['decile_model_score'] = pd.qcut(temp['model_score'], 10, labels=False)

# 2) aggregate sum and count of Exited per (decile, class)
agg = (
    temp
    .groupby(['decile_model_score', 'class_tgt'])['Exited']
    .agg(['sum', 'count'])
    .unstack('class_tgt')  # columns: multiindex (sum, class), (count, class)
)

# separate sum and count tables, fill missing with 0
sum_tbl = agg['sum'].fillna(0)
cnt_tbl = agg['count'].fillna(0)

# 3) cumulative sums over deciles (index order is decile_model_score)
cum_sum = sum_tbl.cumsum()   # cumulative exited by decile × class
cum_cnt = cnt_tbl.cumsum()   # cumulative population by decile × class

# 4) cumulative exited mean per decile × class
pivot_cum_mean = (cum_sum / cum_cnt * 100).round(2)

# 5) cumulative "Total" per decile (across all classes)
decile_sum_total = sum_tbl.sum(axis=1)
decile_cnt_total = cnt_tbl.sum(axis=1)

cum_sum_total = decile_sum_total.cumsum()
cum_cnt_total = decile_cnt_total.cumsum()

total_per_decil_cum = (cum_sum_total / cum_cnt_total * 100).round(2)

pivot_cum_mean['Total'] = total_per_decil_cum

# 6) final "Total" row (overall mean per class and overall)
total_per_class = temp.groupby('class_tgt')['Exited'].mean().round(2) * 100
total_per_class.loc['Total'] = temp['Exited'].mean().round(2) * 100

pivot_cum_mean.loc['Total'] = total_per_class

pivot_cum_mean

/tmp/ipykernel_31935/2734924118.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['decile_model_score'] = pd.qcut(temp['model_score'], 10, labels=False)
/tmp/ipykernel_31935/2734924118.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(['decile_model_score', 'class_tgt'])['Exited']
/tmp/ipykernel_31935/2734924118.py:37: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence thi

class_tgt,ntgt,tgt,tgt_excred,tgt_exp,Total
decile_model_score,,,,,
0,0.00,0.32,0.48,0.16,0.24
1,1.25,0.65,0.87,0.72,0.87
2,1.51,1.24,0.80,1.24,1.20
3,2.15,2.01,1.81,2.02,1.99
4,2.72,3.11,2.76,3.08,2.92
5,4.11,4.44,4.06,4.29,4.23
6,6.16,6.25,6.04,6.14,6.15
7,9.17,9.53,8.82,8.55,9.02
8,14.02,14.10,13.78,13.36,13.82


In [40]:
mean_no_action_class_tgt_json = pivot_cum_mean.to_html(border=0)

In [41]:
mean_no_action_class_tgt_json = mean_no_action_class_tgt_json.replace(
    'class="dataframe"',
    'class="dataframe invert-color"'
)

#### 1.1 Compute the acumulated volume before the exited reachs 10% of the public

In [42]:
import numpy as np

df = df_train.sort_values('model_score').reset_index(drop=True)

target_rate = 0.10  # 10%

# cumulative public and exits
cum_public = np.arange(1, len(df) + 1)
cum_exited = df['Exited'].cumsum().to_numpy()

cum_rate = cum_exited / cum_public  # cumulative exit rate

mask = cum_rate >= target_rate

if not mask.any():
    print("Cumulative exit rate never reaches 10%.")
else:
    # first index where cumulative exit rate >= 10%
    idx = mask.argmax()  # or np.where(mask)[0][0]

    stop_row = df.iloc[idx]
    pct_public_used = (idx + 1) / len(df) * 100
    achieved_rate = cum_rate[idx]
    score_threshold = stop_row['model_score']

    print(f"Public used until cumulative 10% Exited: {pct_public_used:.2f}% of total")
    print(f"Achieved cumulative exit rate: {achieved_rate:.4f}")
    print(f"Score threshold: {score_threshold}")

Public used until cumulative 10% Exited: 82.25% of total
Achieved cumulative exit rate: 0.1000
Score threshold: 0.4465691320023608


In [43]:
def safe_public_until_10pct(y_true, y_score):
    """
    Return % of public (0–100) needed until cumulative Exited reaches 10% of the public.
    If segment never reaches 10% Exited, returns np.nan.
    """
    # ensure we have data
    if len(y_true) == 0:
        return np.nan

    df_seg = pd.DataFrame({
        "y": y_true,
        "score": y_score
    }).dropna()

    if len(df_seg) == 0:
        return np.nan

    # sort by score: highest first
    df_seg = df_seg.sort_values("score", ascending=False).reset_index(drop=True)

    total_public = len(df_seg)
    target_exited = 0.10 * total_public

    cum_exited = df_seg["y"].cumsum().to_numpy()

    # if we never reach 10% of public as Exited, return NaN
    if cum_exited[-1] < target_exited:
        return np.nan

    # first index where cumulative exited >= target_exited
    idx = np.argmax(cum_exited >= target_exited)

    pct_public_used = (idx + 1) / total_public * 100.0
    return round(pct_public_used, 2)

In [44]:
rows = []
groups = ['bcos', 'canal']  # your grouping variables
mdls = ['model_score', 'model_score2']  # example list of model score columns
target = 'Exited'

temp = df_train[df_train['no_action'] == 1]

# ---------- overall ----------
overall_row = {
    "level_type": "overall",
    "n": len(temp),
    "segment": "all",
}

for m in mdls:
    overall_row[f"public_10pct_{m}"] = safe_public_until_10pct(
        temp[target],
        temp[m]
    )

rows.append(overall_row)

# ---------- per group ----------
for grp in groups:
    for level in temp[grp].cat.categories:
        temp_grp = temp[temp[grp] == level]

        row = {
            "level_type": grp,
            "n": len(temp_grp),
            "segment": level,
        }

        for m in mdls:
            row[f"public_10pct_{m}"] = safe_public_until_10pct(
                temp_grp[target],
                temp_grp[m]
            )

        rows.append(row)

public_pct_df = pd.DataFrame(rows)


In [45]:
public_pct_df

,level_type,n,segment,public_10pct_model_score,public_10pct_model_score2
0,overall,24812,all,12.22,12.30
1,bcos,4941,Bco1,12.31,12.43
2,bcos,4874,Bco2,12.21,12.29
3,bcos,5019,Bco3,12.55,12.39
4,bcos,4947,Bco4,12.23,12.21
5,bcos,5031,Bco5,11.93,12.07
6,canal,8384,Abcon,12.45,12.52
7,canal,8235,Jacli,12.30,12.40
8,canal,8193,Pcos,11.96,11.99


In [46]:
# transpose so models are rows and each segment is a column
pct_long = public_pct_df.melt(
    id_vars=['level_type', 'segment'],
    value_vars=[c for c in public_pct_df.columns if c.startswith('public_10pct_')],
    var_name='model',
    value_name='gini'
)
pct_long['model'] = pct_long['model'].str.replace('public_10pct_', '')
pct_matrix = pct_long.pivot(index='model', columns='segment', values='gini')

# order columns: overall 'all' first, then bcos then canal (if present)
cols_order = ['all'] \
    + [seg for grp in groups for seg in public_pct_df[public_pct_df['level_type']==grp]['segment'].tolist()]
cols_order = [c for c in cols_order if c in pct_matrix.columns]
pct_matrix = pct_matrix[cols_order]

pct_no_action_seg_json = pct_matrix.to_html(border=0)

Volume per class_tgt and model_score decile


#### 2.1 Description

In [47]:
### JINJA TEMPLATE
from jinja2 import Template

# Read the template from the template sheet
template_path = "../sandMdlLog/MdlLog_template.html"
with open(template_path, "r") as file:
    sanEda_mdl_template = file.read()

# Create a Jinja2 template object

template = Template(sanEda_mdl_template)

# Render the template with the data
rendered_html = template.render(
    title=pipeline_parameters['model_name'],
    description_general = pipeline_parameters['description_general'],
    description_specific = pipeline_parameters['description_specific'],
    tbl_version = pipeline_parameters['tbl_version'],
    inf_version = pipeline_parameters['inf_version'],
    mdl_version = pipeline_parameters['mdl_version'],
    init_params_txt = str(pipeline_parameters),
    tgt_neg_criteria = '10',
    ntgt_aprv_criteria = '2',
    gini_obs_graph_json = gini_obs_graph_json,
    gini_obs_inf_graph_json = gini_obs_inf_graph_json,
    gini_no_action_graph_json = gini_no_action_graph_json,
    gini_train = gini_train,
    gini_test = gini_test,
    gini_oot = gini_oot,
    gini_noaction = gini_no_action,
    tbl_vol_seg = vol_seg_json,
    tbl_gini_obs_seg = gini_obs_seg_json,
    tbl_gini_obs_inf_seg = gini_obs_inf_seg_json,
    tbl_gini_no_action_seg = gini_no_action_seg_json,
    tbl_vol_class_tgt = vol_class_tgt_json,
    tbl_vol_no_action_class_tgt = vol_no_action_class_tgt_json,
    tbl_mean_class_tgt = mean_class_tgt_json,
    tbl_mean_no_action_class_tgt = mean_no_action_class_tgt_json,
)

# Save the rendered HTML to a file
with open(f"report_{pipeline_parameters['model_name']}.html", "w") as file:
    file.write(rendered_html)